# PoC: WeCookio multiagent Agno Agents

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Coordinate flow with delegation

### Installing Required Packages with uv

In [1]:
# Install required packages using uv
!uv add ipykernel agno==1.7.9 boto3 botocore pyyaml ipywidgets prompt-template sqlalchemy duckduckgo-search

Resolved 224 packages in 0.76ms
Audited 213 packages in 0.07ms


### Import Libraries and Load Configuration

In [2]:
import yaml
import logging
from typing import Iterator
from agno.agent import Agent
from agno.models.aws import AwsBedrock
from agno.team import Team, TeamRunResponse
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.utils.pprint import pprint_run_response

print("Libraries imported successfully!")

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


Libraries imported successfully!


### Loading AWS Enviroment variables

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [4]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "agno/latest/"
# Load the integration configuration
with open(prefix+'config/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/config.yaml', 'r') as file:
    agno_config = yaml.safe_load(file)
with open(prefix+'config/agents-teams.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/tasks-teams.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(agno_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

/home/e2its/dev/weCookio/researches
Configuration files loaded successfully!
Integration config keys: ['aws_bedrock', 'database', 'api_endpoints']
Crew Orchestration keys: ['crew', 'agents', 'tasks', 'integration_config', 'model_assignment', 'execution', 'memory', 'input_schema', 'output_schema', 'cost_optimization', 'performance_metrics', 'quality_standards', 'monitoring', 'example_input', 'example_output']
Agents config keys: ['Culinary Coordinator', 'Dietary Compliance Specialist', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Quality Assurance Chef']
Tasks config keys: ['Enhance Recipe']


###  Model Selection based on complexity

In [5]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

High complexity model: bedrock/amazon.nova-premier-v1:0
Medium complexity model: bedrock/amazon.nova-pro-v1:0
Low complexity model: bedrock/amazon.nova-lite-v1:0
Ultra-Low complexity model: bedrock/amazon.nova-micro-v1:0


### Create Agents with Model-Specific Configuration

In [6]:
def create_agent_with_model(agent_config, model_config, **kwargs):
    """Create an agent with specific model configuration"""
    llm = AwsBedrock(
        id=model_config["model_id"].split("/")[-1],
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
    )

    args = {}
    args["model"]=llm
    args['role']=agent_config.get("role", None)
    args['goal']=agent_config.get("goal", None)
    args['instructions']=agent_config.get("prompt", None)
    args["description"]=agent_config.get("backstory", None)
    args["name"]=agent_config.get("name", None)
    args["add_datetime_to_instructions"]=True
    
    return Agent(**args)

def print_agent_info(agent) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.name}\n"
        f"Agent Description: {agent.description}\n"
        f"Using model: {agent.model}\n"
        f"Role: {agent.role}\n"
        f"Goal: {agent.goal}\n"
        f"Instructions: {agent.instructions}\n"
    )
    print("\n")

In [7]:
# Global metrics storage
def reset_execution_metrics():
    global execution_metrics
    execution_metrics = {
        'audio_tokens': [0],
        'cache_write_tokens': [0],
        'cached_tokens': [0],
        'completion_tokens': [0],
        'input_audio_tokens': [0],
        'input_tokens': [0],
        'output_audio_tokens': [0],
        'output_tokens': [0],
        'prompt_tokens': [0],
        'reasoning_tokens': [0],
        'time': [0],
        'total_tokens': [0]
    }
    return execution_metrics

def get_execution_metrics():
    return execution_metrics

def set_execution_metrics(kwargs):
    """Safely extract metrics from kwargs"""
    try:
        execution_metrics['audio_tokens'][0] += kwargs.get('audio_tokens')[0]
        execution_metrics['cache_write_tokens'][0] += kwargs.get('cache_write_tokens')[0]
        execution_metrics['cached_tokens'][0] += kwargs.get('cached_tokens')[0]
        execution_metrics['completion_tokens'][0] += kwargs.get('completion_tokens')[0]
        execution_metrics['input_audio_tokens'][0] += kwargs.get('input_audio_tokens')[0]
        execution_metrics['input_tokens'][0] += kwargs.get('input_tokens')[0]
        execution_metrics['output_audio_tokens'][0] += kwargs.get('output_audio_tokens')[0]
        execution_metrics['output_tokens'][0] += kwargs.get('output_tokens')[0]
        execution_metrics['prompt_tokens'][0] += kwargs.get('prompt_tokens')[0]
        execution_metrics['reasoning_tokens'][0] += kwargs.get('reasoning_tokens')[0]
        execution_metrics['time'][0] += kwargs.get('time')[0]
        execution_metrics['total_tokens'][0] += kwargs.get('total_tokens')[0]
        return execution_metrics
    except Exception as e:
        print(f"⚠️  Warning: Could not extract metrics: {e}")
    return execution_metrics

In [8]:

agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
culinary_coordinator_agent.tools = [DuckDuckGoTools()]
print_agent_info(culinary_coordinator_agent)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent)


Created agent: Culinary Coordinator
Agent Description: Executive chef with 20+ years experience in dietary adaptations and team coordination
Using model: AwsBedrock(id='amazon.nova-lite-v1:0', name='AwsBedrock', provider='AwsBedrock', supports_native_structured_outputs=False, supports_json_schema_outputs=False, _tool_choice=None, system_prompt=None, instructions=None, tool_message_role='tool', assistant_message_role='assistant', aws_sso_auth=False, aws_region=None, aws_access_key_id=None, aws_secret_access_key=None, session=None, max_tokens=2000, temperature=0.4, top_p=0.9, stop_sequences=None, request_params=None, client=None, async_client=None, async_session=None)
Role: Master Chef & Project Manager
Goal: Orchestrate recipe adaptation ensuring maximum culinary excellence and strict dietary compliance
Instructions: You are the master chef for weCookio's culinary adaptation system.

**CRITICAL: Zero tolerance for dietary restriction violations - this is your top priority.**
**CRITICAL:

### Create a Flow

In [9]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Leche",
        "Huevo",
        "Ternera",
        "Maiz",
        "Gluten",
        "Arroz",
        "Ajo"
    ],
    "exclusions": ["Pimiento"],
    "preferences": [],
    "output_language": "spanish",
    "country": "Spain"
}

In [10]:
## Support functions
from prompt_template import PromptTemplate, InvalidTemplateKeysError, MissingTemplateValuesError, TemplateSerializationError

def variable_injection(task:dict, recipe_context:dict):
    try:
        prompt = PromptTemplate(name=task.get("name"), template=f'''{task.get("description")}''').to_string(**recipe_context)
    except MissingTemplateValuesError as e:
        print(f"Missing values: {e.missing_values}")  # {'name'}
    except InvalidTemplateKeysError as e:
        print(f"Invalid keys: {e.invalid_keys}")  # {'name'}
    except TemplateSerializationError as e:
        print(f"Serialization error: {e.message}")  # 'Invalid template string'
    except Exception as e:
        print(f"Unexpected error: {e}")  # 'Unexpected error'
    return prompt


In [11]:
tasks = {
    "enhance_recipe": {
        "description": f'''{variable_injection(tasks_config['Enhance Recipe'], recipe_request)}''',
        "dependencies": []
    }
}
print(tasks['enhance_recipe'].get("description")) 

Enhance the recipe with the following details:
## RECIPE CONTEXT
- **Recipe Name**: Canelones de ternera XXL al Pedro Ximenez
- **Servings**: 4
- **Dietary Restrictions**: ["Leche", "Huevo", "Ternera", "Maiz", "Gluten", "Arroz", "Ajo"], ["Pimiento"]
- **Preferences**: []
- **Target Country**: Spain
- **Output Language**: spanish

## TASK OBJECTIVE
Transform the requested recipe to be 100% compliant with dietary restrictions and adapted to preferences 
while maximizing culinary experience and preserving authentic flavor/texture characteristics and provide the expected output indicated bellow.

**Expected output:**
- Normalize the recipe and instructions to the target Spain
- Prepare a separate section with the expected changes in ingredients and their amounts to achieve the desired result.
- Return the detailed step by step instructions in the target Spain format and language spanish
- Validation status for Quality checks


In [12]:
# Coordinate Team processing

agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
llm = AwsBedrock(
    id=model_config["model_id"].split("/")[-1],
    temperature=model_config.get("model_kwargs").get("temperature", 0.6),
    top_p=model_config.get("model_kwargs").get("top_p", 0.9),
    max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
)
wecookio_team = Team(
    name=agent_type,
    description=agent_config.get("backstory"),
    instructions=agent_config.get("prompt"),
    expected_output=agent_config.get("expected_output"),
    success_criteria=agent_config.get("success_criteria"),
    model=llm,
    mode="collaborate",
    add_member_tools_to_system_message=False,  # This can be tried to make the agent more consistently get the transfer tool call correct
    enable_agentic_context=True,  # Allow the agent to maintain a shared context and send that to members.
    share_member_interactions=True,  # Share all member responses with subsequent member requests.
    show_members_responses=True,
    show_tool_calls=True,
    markdown=True,
    debug_mode=True,
    members=[ingredient_substitution_expert_agent, culinary_experience_optimizer_agent, quality_assurance_chef_agent],
)


### Run the Flow

In [13]:
try:
    result : TeamRunResponse = wecookio_team.run(f'''{tasks['enhance_recipe'].get("description")}''')
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

DEBUG Processing tools for model

DEBUG Added tool set_shared_context

DEBUG Added tool run_member_agents

DEBUG *** Team Run Start: 652bc1e7-ddc7-4811-8562-70fe22d329e9 ***

DEBUG ******************* Mode: 'collaborate' ********************

...

DEBUG ************************  METRICS  *************************

DEBUG ----------------- AwsBedrock Response End ------------------

DEBUG Added RunResponse to Memory

DEBUG --**-- Logging Team Run

DEBUG **** Team Run End: 652bc1e7-ddc7-4811-8562-70fe22d329e9 ****

In [14]:
pprint_run_response(result, markdown=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│ To enhance the recipe "Canelones de ternera XXL al Pedro Ximenez" while adhering to the dietary restrictions    │
│ and maximizing culinary experience, I need to coordinate the team. I will first set the shared context with the │
│ recipe details and dietary restrictions. Then, I will run the team members to get their inputs on ingredient    │
│ substitutions, culinary enhancements, and quality assurance. Finally, I will compile the responses and ensure   │
│ the final recipe meets the success criteria. I have the responses from all the agents. Now, I will analyze the  │
│ responses to ensure they meet the success criteria, focusing on dietary restrictions, flavor preservation, and  │
│ culinary experience. I will then compile the final enhanced recipe with all the necessary details.              │
│                                                                                                                 │
│ Enhanced Recipe: Canelones de ternera XXL al Pedro Ximenez                                                      │
│                                                                                                                 │
│ Porciones: 4                                                                                                    │
│                                                                                                                 │
│ Restricciones Dietéticas: Leche, Huevo, Ternera, Maíz, Gluten, Arroz, Ajo, Pimiento                             │
│                                                                                                                 │
│                                                  Ingredientes:                                                  │
│                                                                                                                 │
│ Para el relleno:                                                                                                │
│                                                                                                                 │
│  • 300g de pollo (en lugar de ternera)                                                                          │
│  • 1 taza de champiñones finamente picados (en lugar de pimiento)                                               │
│  • 1 cebolla pequeña, picada                                                                                    │
│  • 2 dientes de chalote (en lugar de ajo)                                                                       │
│  • 1 taza de salsa de tomate sin gluten                                                                         │
│  • 1/2 taza de vino Pedro Ximenez                                                                               │
│  • 1 cucharada de tomillo fresco                                                                                │
│  • Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│ Para la bechamel:                                                                                               │
│                                                                                                                 │
│  • 3 cucharadas de harina de garbanzos (en lugar de ha